## Old Functions and Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# Averaging Datasets

In [ ]:
# ## TR6 -- to get time_zero

# ddir = r"E:\atully\arpes_data\2023_May\May13\Bi2Se3\TR6"
# files = []
# files = [f"TR6_001_{i}.h5" for i in range(1, 8)]

# ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
# ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
# for file in tqdm(files):
#     data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
#     ARPES_DATA[file] = tr_functions.ArpesData(
#         data=data, theta=kx, phi_or_time=ky, energy=energy
#     )
#     ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# average_timescans(files, ddir, "TR6_Ali_avg.h5")

# Load Dataset

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\Bi2Se3"
files = []
files = [f"TR3eV_019.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

In [ ]:
# time_zero = 37.79  # TR6

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 50  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None

x_bin = 1
y_bin = 1

In [ ]:
title = f"Bi<sub>2</sub>Se<sub>3</sub> Pump Effect: {files[0]}"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]",
xaxis_title = f"Delay"
yaxis_title = f"E<sub>K</sub> (eV)"

In [ ]:
## Load Data ##

# xaxis_ps = False
# xaxis_ps = True

x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_dataclass=ad,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    # ylim=ylim,
    ylim=(
        ad.energy[57],
        ad.energy[1007],
    ),  # get rid of zero padding on datasets
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

# # Convert mm to ps
# if xaxis_ps and np.max(x_2d) > 37:
#     x_2d = tr_functions.mm_to_ps(x_2d, time_zero)

## Plot Data
fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=800,
    dtick_y=0.1,
)

fig.add_trace(
    go.Heatmap(
        x=x_2d,
        y=y_2d,
        z=analysis_functions.norm_data(d_2d),
        coloraxis="coloraxis",
    )
)

# for h in [3.3, 3.545]:
#     fig.add_hline(y=h, line=dict(color="black", width=1, dash="dash"))

# fig.update_coloraxes(colorscale="greys", showscale=False)

fig.show()

In [ ]:
## Integrated over energy ##

xlim = None
# ylim = None
ylim = (3.3, 3.545)

## Get slice
x_1d, row = tr_functions.get_1d_y_slice(
    x=x_2d,
    y=y_2d,
    data=d_2d,
    xlims=xlim,
    y_range=ylim,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"Average over {ylim} eV",
    xaxis_title="Delay (mm)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

fig.add_trace(go.Scatter(x=x_1d, y=row, name="data"))

## For After Fit
fig.add_trace(go.Scatter(x=x, y=fit.eval(x=x_1d), name="fit"))

components = fit.eval_components(x=x_1d)
for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        # x=fit.params[f"center"].value,
        # y=fit.eval(x=fit.params[f"{model_name}center"].value),
        xref="x domain",
        yref="y domain",
        x=0.01,
        y=0.99,
        showarrow=False,
        text=f'Peak center: {fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} mm<br>FWHM: {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params["sigma"].value)):.3f} +/- {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr)):.4f} ps',
        font=dict(size=18),
    )

fig.show()

In [ ]:
## Plot Residuals
fig = tr_functions.thesis_fig(
    title=f"{title}",
    xaxis_title=xaxis_title,
    yaxis_title="Residuals",
    equiv_axes=False,
    gridlines=False,
    height=400,
    width=900,
)

fig.add_trace(go.Scatter(x=x_1d, y=row - fit.eval(x=x_1d), name="fit"))
fig.add_hline(y=0, line=dict(dash="dash"))

In [ ]:
## Attempt to manually build convolved model: This is the built-in model ##

import lmfit as lm

offset_type = "constant"
x = x_1d
data = row

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# gaussian = fitting_functions.make_gaussian(
#     num="A",
#     amplitude=1,
#     center=37.8,
#     sigma=0.05,
#     include_exp_decay=True,
#     gamma=19,
#     # lock_sigma=True,  # sigma should be based on fwhm of BiSe, but the pump profile changes
# )

gaussian = lm.models.ExponentialGaussianModel()
gaussian = lm.models.GaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

params["center"].value = 34.8
params["amplitude"].value = 1
params["sigma"].value = 0.05
# params["gamma"].value = 20
# params["gamma"].max = 200
# params["gamma"].min = 1

# params["iA_gamma"].value = 20
# params["iA_gamma"].max = 200
# params["iA_gamma"].min = 1


fit = full_model.fit(row, x=x_1d, params=params)

fit.plot()

In [ ]:
fit.params

In [ ]:
tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params["sigma"].value))